# Document Normalizer annotator notebook

# Set up Colab environment

In [ ]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

# Start Spark NLP session

In [ ]:
# Import Spark NLP
from sparknlp.base import *
from sparknlp.annotator import *

def start():
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[4]") \
        .config("spark.driver.memory", "8G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.7.1") \
        .config("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
        .config("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
    return builder.getOrCreate()

spark = start()

In [ ]:
spark

# Regex Tokenizer annotator

In [ ]:
from pyspark.ml import Pipeline
from pyspark.sql.types import StringType
from sparknlp.base import *
from sparknlp.annotator import *
import sparknlp

content = "1. T1-T2 DATE**[12/24/13] $1.99 () (10/12), ph+ 90%"
pattern = "\\s+|(?=[-.:;*+,$&%\\[\\]])|(?<=[-.:;*+,$&%\\[\\]])"

df = spark.createDataFrame([content], StringType()).withColumnRenamed("value", "text")

df.show()

documentAssembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

sentenceDetector = SentenceDetector() \
      .setInputCols(["document"]) \
      .setOutputCol("sentence")

regexTokenizer = RegexTokenizer() \
      .setInputCols(["sentence"]) \
      .setOutputCol("regexToken") \
      .setPattern(pattern) \
      .setPositionalMask(False)

docPatternRemoverPipeline = \
  Pipeline() \
    .setStages([
        documentAssembler,
        sentenceDetector,
        regexTokenizer])

ds = docPatternRemoverPipeline.fit(df).transform(df)

ds.show(10, False)